# Topik Modeling Dengan Python

topic modeling merupakan suatu pendekatan untuk menganalisis kumpulan dokumen berbentuk teks dan mengelompokkan menjadi beberapa topik. Pendekatan tersebut masuk dalam pendekatan Clustering dalam studi Machine Learning. Adapun tahap-tahapnya yaitu : 
<ol>
    <li>Crawling Data</li>
    <li>Preprocessing Data</li>
    <li>LSA</li>
</ol>

## Crawling Data

Crawling data adalah suatu teknik untuk mengumpulkan data secara cepat dengan menggunakan url sebagai target data yang akan dikumpulkan. Untuk mengumpulkan data bisa menggunakan berbagai tools atau library yang ada, salah satunya adalah Scrappy. Scrapy adalah framework dari python yang berspesialis dalam melakukan web scraping dalam sekala besar, untuk menggunakan scrapy pertama kita install dahulu Scrapy dengan menggunakan pip

### Install Library

Library yang perlu diinstal untuk crawling data ada dua yaitu, Scrapy dan nltk

In [ ]:
pip install Scrapy

In [ ]:
pip install --user -U nltk

### Import Library

Sesudah install library yang dibutuhkan, selanjutnya kita import librarynya (untuk re sudah terinstall otomatis ketika install python)

In [2]:
import scrapy
import nltk
import re

### Melakukan Crawling

Sesudah import library yang dibutuhkan, selanjutnya melakukan tahap crawling. Disini tahap Crawl saya simpan di class QuotesSpider. Variabel start_urls berfungsi untuk menampung target url, dimana start_url akan mendapatkan data dari tahap looping "for page in range(1,10)". Function parse memiliki peran melakukan scrap pada element html mana, sedangkan function parse_detail memiliki peran untuk menargetkan secara spesifik seperti : 
<ul>
    <li>Mengambil text htmlnya atau mengambil Linknya</li>
    <li>Membuang elemen yang tidak digunakan</li>
    <li>Mereplace kata yang tidak digunakan dengan kata yang ingin digunakan</li>
</ul>

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = []
    def __init__(self):
        url = 'https://pta.trunojoyo.ac.id/welcome/index/'
        for page in range(1,10):
            self.start_urls.append(url+str(page))

    def parse(self, response):
        for detail in response.css('a.gray.button::attr(href)'): 
            yield response.follow(detail.get(), callback = self.parse_detail)

    def parse_detail(self, response):
        for data in response.css('#content_journal > ul > li'):
            yield{
                'Judul': data.css('div:nth-child(2) > a::text').get(),
                'Penulis': data.css('div:nth-child(2) > span::text').get().replace('Penulis : ', ''),
                'Dospem 1': data.css('div:nth-child(3) > span::text').get().replace('Dosen Pembimbing I : ', ''),
                'Dospem 2': data.css('div:nth-child(4) > span::text').get().replace('Dosen Pembimbing II :', ''),
                'Abstraksi': data.css('div:nth-child(2) > p::text').get().replace('\n\n|\n','').replace('ABSTRAK', ''),
                'Abstraction': data.css('div:nth-child(4) > p::text').get().replace('\n\n|\n','').replace('ABSTRACT', ''),
                'Link Download': data.css('div:nth-child(5) > a:nth-child(1)::attr(href)').get().replace('.pdf-0.jpg', '.pdf'),
            }

Silahkan save codenya dan buka cmd, pastikan terbuka di folder yang ada file scrapingnya. Kemudian jalankan perintah ini di cmd untuk memproses dan menyimpan ke csv "scrapy runspider namaFile.py -o namaFileKetikaDiSaveUlang.csv"

## Preprocessing Data

Preprocessing Data adalah suatu teknik untuk merubah data mentah atau raw data menajdi informasi yang bersih dan agar bisa digunakan untuk pengolahan lanjutan pada data mining. Pada pembahasan ini Preprocessing Data akan dilakukan dalam 2 tahap, yaitu :
<ol>
    <li>Stop Word</li>
    <li>Cleaning Data</li>
</ol>

### Install Library

Library yang perlu diinstall untuk melakukan preprocesing data ada dua yaitu, nltk dan scikit learn

In [ ]:
pip install --user -U nltk

In [ ]:
pip install -U scikit-learn

### Import Library

Sesudah install, kita import library yang dibutuhkan (untuk string udah otomatis terinstall)

In [6]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

### Melakukan Preproces

#### 1. Stop Word

Stop Word adalah tahap untuk menghilangkan kata yang tidak memiliki arti, seperti preposisi, konjungsi, dan lain sebagainya. Contoh kata yang dihilangkan dari Stop Word adalah yang, di, ke, dan lainnya. Tanpa perlu berlama-lama mari langsung kepada tahap kodingnya, pertama tama kita load data yang sudah kita crawling tadi. Karena tadi hasil yang saya save dengan nama **crawlingpta.csv** maka pada saat load dengan pandas yang saya tuju adalah file **crawlingpta.csv**

In [7]:
jurnal = pd.read_csv('crawlingpta.csv')

Sesudah meload data selanjutnya memilih kolom yang ingin di proses, disini saya akan memproses kolom **abstraksi**, dan pada kolom itu juga saya akan menghilangkan angka yang akan mengganggu. Tahap ini juga termasuk dalam bagian Cleaning Data, tahap ini saya lakukan di awal karena kalau udah masuk ke stop word akan susah di proses. Untuk melakukannya saya buat function yang bernama **remove_number** dan di function ini akan mengembalikan nilai berupa text dimana jika ada angka akan dihapus, dan ketika memanggil kolom dikasih apply dan memanggil functionnya

In [8]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

pre_abstrak = jurnal['Abstraksi'].apply(remove_number)
pre_abstrak

0      Mayarakat relawan Indonesia (MRI) Surabaya ter...
1      Identifikasi atribut pejalan kaki merupakan sa...
2      Skripsi ini bertujuan untuk menganalisis penti...
3      \n\nTujuan utama dari penelitian ini adalah un...
4      Penelitian ini bertujuan untuk dapat mengetahu...
                             ...                        
847    Penelitian ini bertujuan untuk mengetahui peng...
848    Tujuan penelitian ini adalah untuk mengetahui ...
849     \nJenis penelitian ini merupakan penelitian e...
850    Ach. Fatahillah, NIM  Program Studi Sosiologi,...
851    \nBayu Krisnatama, “Analisis Pendapatan dan Da...
Name: Abstraksi, Length: 852, dtype: object

Kemudian langkah sebelum memasuki stop word adalah harus tokenize kalimat dahulu, tokenize adalah proses untuk membagi kalimat ke dalam bagian bagian tertentu

In [9]:
word_tokens = pre_abstrak.apply(word_tokenize)
word_tokens

0      [Mayarakat, relawan, Indonesia, (, MRI, ), Sur...
1      [Identifikasi, atribut, pejalan, kaki, merupak...
2      [Skripsi, ini, bertujuan, untuk, menganalisis,...
3      [Tujuan, utama, dari, penelitian, ini, adalah,...
4      [Penelitian, ini, bertujuan, untuk, dapat, men...
                             ...                        
847    [Penelitian, ini, bertujuan, untuk, mengetahui...
848    [Tujuan, penelitian, ini, adalah, untuk, menge...
849    [Jenis, penelitian, ini, merupakan, penelitian...
850    [Ach, ., Fatahillah, ,, NIM, Program, Studi, S...
851    [Bayu, Krisnatama, ,, “, Analisis, Pendapatan,...
Name: Abstraksi, Length: 852, dtype: object

Langkah selanjutnya adalah Stop Word. Karena disini saya menggunakan nltk maka harus menentukan dahulu bahasa yang digunakan untuk menentukan bahasa menggunakan **stopwords.words('indonesian')**. Kemudian jika dirasa list stop word masih ada yang kurang maka kita bisa menambahkan sendiri dengan cara membuat list kata yang tidak ada di stop word kemudian kita extend dengan list yang kita buat sendiri **stop_words.extend(list)**

In [ ]:
stop_words = stopwords.words('indonesian')
list = ['a','aajaran','aanslag','aatau','ah','abstak','abstrack','abstract','abstrak','z']
stop_words.extend(list)
after = [[w for w in temp if w not in stop_words] for temp in word_tokens]
for i in after:
    print(i)

Untuk logika pada saat stop word sendiri sebagai berikut. Pertama kita set bahasa stop words yang digunakan yaitu **indonesian**. Jika ada list stop words yang tidak ada pada stop words yang disediakan oleh nltk, kita bisa menambahkannya dengan cara membuat list kata yang mau dihilangkan kemudian pada stop wordsnya di extend dengan list yang menyimpan list kata yang ingin dihapus. Kemudian logika untuk perulangannya yaitu ini akan dilooping kata yang ada di dalam nested array, maka kita lakukan 2 kali perulangan. Pertama untuk melooping yg ada di dalam nestednya dengan dikasih logika percabangan jika katanya tidak ada pada list stop wordsnya maka akan masuk, dan yang kedua untuk menentukan list mana yang akan di looping.

#### 2. Cleaning Data

Cleaning Data adalah proses untuk membersihkan data yang ada menjadi data yang bisa diolah. Data yang dibersihkan seperti missing value atau data kosong, karakter asing, menghilangkan angka, dan lain sebaginaya. Untuk proses penghilangan angka sudah dilakukan ketika memilih tabel **abstraksi**, maka sekarang tinggal menghilangkan karakter asing dan sekawannya. Untuk melakukan itu kita bisa menggunakan library string.punctuation. Dimana ia akan menghilangkan karakter asing yang ada

In [ ]:
clearData = [[w for w in z if w not in string.punctuation and w.isalpha()] for z in after] 
for i in clearData:
    print(i)

Logika dari code diatas sama seperti proses stop words dimana dilakukan perulangan nested looping untuk mengecek katanya, jika terdeteksi kata itu ada pada **string.punctuation** maka tidak dimasukan

## Pemodelan Dengan LSA

LSA merupakan metode yang memanfaatkan model statistik matematis untuk menganalisa struktur semantik suatu teks. LSA bisa digunakan untuk menilai esai dengan mengkonversikan esai menjadi matriks-matriks yang diberi nilai pada masing-masing term untuk dicari kesamaan dengan term referensi.

### Install Library

Library yang butuh untuk diinstall adalah scikit learn

In [ ]:
pip install -U scikit-learn

### Import Library

Sesudah insall selanjutnya import TfidfVectorizer dan TruncatedSVD dari library scikit learn atau sklearn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

### Membuat Document Term Matrix (DTM)

Document Term Matrix adalah matriks matematis yang menggambarkan frekuensi istilah yang muncul dalam kumpulan dokumen, dengan DTM kita bisa dengan mudah untuk menentukan jumlah kata individual untuk setiap dokumen atau untuk semua dokumen. Misalkan untuk mengetahui kata mana yang lebih sering muncul dalam kumpulan dokumen dan menggunakan informasi tersebut untuk menentukan kata mana yang lebih mungkin “mewakili” dokumen tersebut. Nilai dari DTM sendiri menggunakan nilai dari TF-Idf. Beberapa poin penting yang perlu diperhatikan:
<ol>
    <li>LSA pada umumnya diimplementasikan dengan menggunakan nilai TF-Idf dan tidak dengan Count Vectorizer</li>
    <li>Nilai parameter max_feature bergantung pada daya komputasi</li>
    <li>Nilai default untuk min_df dan max_df agar program dapat bekerja dengan baik</li>
    <li>Bisa menggunakan nilai ngram_range yang berbeda</li>
</ol>
Setelah mengetahui poin penting, berikut code untuk cari Tf-IDF

In [51]:
def dummy(doc):
    return doc
vect = TfidfVectorizer(tokenizer=dummy, lowercase=False, max_features=1000)
vect_text = vect.fit_transform(clearData)
vect_text_tranpose = vect_text.transpose()
df = pd.DataFrame(vect_text_tranpose.toarray())

Setelah kita set Tf-idf dari data, kita cek dahulu apakah baris dan kolom sesuai dengan kata dan dokumennya menggunakan fungsi **shape**. Berikut adalah implementasinya dengan 5 nilai Tf-idf kata per dokumen

In [52]:
print(vect_text.shape)
print(df.head(5))

(852, 1000)
   0    1         2    3    4         5         6    7    8    9    ...  842  \
0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
1  0.0  0.0  0.000000  0.0  0.0  0.056172  0.111792  0.0  0.0  0.0  ...  0.0   
2  0.0  0.0  0.034202  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
3  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   
4  0.0  0.0  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.0  0.0  ...  0.0   

   843  844  845  846  847  848  849  850  851  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 852 columns]


Setelah kita cek ukurannya, berikut adalah hasil kata yang paling sering muncul dan kata yang paling jarang muncul berdasarkan nilai Tf-idf diatas. Semakin kecil nilainya maka dia sering digunakan, sedangkan jika besar nilainya maka dia jarang digunakan.

In [54]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['penelitian'])
print(dd['shift'])

penelitian shift
1.1434616265434774
6.650147258823569


Dapat dilihat bahwa kata penilitian adalah kata paling sering muncul, sedangkan shift adalah kata paling jarang muncul

## Proses Latent Semantic Analysis (LSA)

Setelah membuat DTM, langkah selanjutnya adalah proses Latent Semantic Analysis (LSA). Namun sebelum melakukan LSA ada tahap untuk pengurungan dimensi agar bisa menemukan topik laten yang menangkap hubungan antara kata dan dokumen, pengurangan dimensi ini bisa dilakukan dengan cara Singular Value Decomposition (SVD) seperti dibawah ini

### Singular Value Decomposition (SVD)

Singular Value Decomposition (SVD) adalah teknik pada aljabar linear yang memfaktorkan sembarang matrix menjadi 3 matrix yang berbeda, yaitu:
<ul>
    <li>U = Matrix kolom ortogonal</li>
    <li>V = Matrix baris ortogonal</li>
    <li>S = Satu Singular Matrix</li>
</ul>
Sementara itu ada yang namanya Truncated singular value decomposition (SVD), dimana Truncated SVD merupakan kunci untuk mendapatkan topik dari kumpulan dokumen yang diberikan. Rumus dari Truncated SVD kurang lebih seperti ini $A=U S V^{T}$, dimana :
<ul>
    <li>A mewakili document-term matrix, dengan nilai berbasis hitungan yang ditetapkan di antara setiap dokumen dan pasangan kata. Matrix tersebut memiliki dimensi n x m, dengan n mewakili jumlah dokumen dan m mewakili jumlah kata.</li>
    <li>U mewakili document-topic matrix. Pada dasarnya, nilainya menunjukkan kekuatan hubungan antara setiap dokumen dan topik turunannya. Matriks memiliki n x r dimensi, dengan n mewakili jumlah dokumen dan r mewakili jumlah topik.</li>
    <li>S mewakili matriks diagonal yang mengevaluasi "Strength" setiap topik dalam kumpulan dokumen. Matrix memiliki r x r dimensi, dengan r mewakili jumlah topik.</li>
    <li>V mewakili word-topic matrix. Nilai-nilainya menunjukkan kekuatan asosiasi antara setiap kata dan topik yang diturunkan. Matrix tersebut memiliki dimensi m x r, dengan m mewakili jumlah kata dan r mewakili jumlah topik.</li>
</ul>

![Visualisasi Truncated SVD](visualisasi.png)

In [55]:
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)
lsa_top=lsa_model.fit_transform(vect_text)

print(lsa_top)
print(lsa_top.shape)

[[ 0.22321048 -0.2101906   0.05076316 ... -0.00128485 -0.13422382
  -0.03937861]
 [ 0.13770476 -0.04925036  0.0302596  ... -0.16023675 -0.04645213
  -0.03396618]
 [ 0.05777609 -0.06899392 -0.04774173 ...  0.051956   -0.06244748
   0.00590277]
 ...
 [ 0.22765412 -0.18006729  0.10508355 ... -0.00507767 -0.07534519
   0.00126922]
 [ 0.18020879 -0.16760855 -0.12071342 ...  0.04173434 -0.02178306
   0.02979276]
 [ 0.15163765 -0.18814718 -0.03595521 ... -0.02328596  0.08594999
   0.01027479]]
(852, 10)


In [56]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  22.321047797254273
Topic  1  :  -21.019060233823218
Topic  2  :  5.076316304717713
Topic  3  :  2.657094937469628
Topic  4  :  -12.830867978990417
Topic  5  :  -5.031788650287868
Topic  6  :  5.673181047889446
Topic  7  :  -0.12848547610313046
Topic  8  :  -13.422382119489873
Topic  9  :  -3.937861323781596


In [57]:
print(lsa_model.components_.shape)
print(lsa_model.components_)

(10, 1000)
[[ 0.01722975  0.02354554  0.02214484 ...  0.01009565  0.01052395
   0.01259088]
 [ 0.0022534  -0.01911612 -0.0278433  ... -0.01461903 -0.01861643
  -0.0195887 ]
 [ 0.00785405 -0.02270544 -0.0220577  ... -0.00424094 -0.02008752
  -0.00907963]
 ...
 [-0.00597358 -0.00183798  0.00017789 ... -0.00530835  0.01268106
  -0.00720262]
 [-0.01487477 -0.01487835 -0.00061892 ...  0.00745174  0.01732684
   0.00834296]
 [ 0.02119781  0.00959129  0.01182404 ... -0.01914525  0.01066964
  -0.00475969]]


### Mengekstrak Topic dan Term

Setelah dilakukan Truncated Matrix, sekarang kita dapat melakukan ekstrak topik dokumen. Pada percobaan kali ini, dilakukan extrak sebanyak 10 topik.

In [58]:
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
siswa pembelajaran penelitian data media hasil kelas Penelitian uji metode 

Topic 1: 
pembelajaran siswa media valid perangkat kelas pengembangan ahli materi model 

Topic 2: 
karyawan kinerja kerja berpengaruh signifikan variabel positif nilai perusahaan PT 

Topic 3: 
siswa kemampuan kelas berpikir konsep kritis pemecahan tes SMP indikator 

Topic 4: 
hukum pidana Nomor siswa Tahun Pasal tindak perbuatan ayat hak 

Topic 5: 
garam jagung produk air kualitas kadar produksi daun faktor nilai 

Topic 6: 
the of The in and to that anak study novel 

Topic 7: 
garam the media of air The in hukum and kadar 

Topic 8: 
beli jual the of Desa in The akad Islam siswa 

Topic 9: 
perangkat pembelajaran garam LKK RPP the model Lembar Perangkat beli 

